In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate

import pandas as pd

import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import linear_model
from scipy import stats as sps
from sklearn.feature_selection import SelectFromModel
from IPython.display import display


locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
GAMMA = 1/7.5

In [2]:
def dS(beta, S, I, N):
    return -beta * S * I / N
def dI(beta, gamma, S, I, N):
    return beta * S * I / N - gamma * I
def dR(gamma, S, I, N):
    return gamma * I

def dY(x, t):
    N = 10000
    S = x[0]
    I = x[1]
    R = x[2]
    beta = 2 * GAMMA
    gamma = GAMMA
    if t>20 and beta == 2 * GAMMA:
        beta = 1.5 * GAMMA
    if t>40 and beta == 1.5 * GAMMA:
        beta = 0.7 * GAMMA
    if t>60 and beta == 0.7 * GAMMA:
        beta = 0.4 * GAMMA
    #if t>80 and beta == 0.7 * GAMMA:
    #    beta *= 0. * GAMMA

    dx0 = dS(beta, S, I, N)
    dx1 = dI(beta, gamma, S, I, N)
    dx2 = dR(gamma, S, I, N)
    return np.array([dx0, dx1, dx2])

In [3]:
np.random.seed(1234)
plt.close('all')
dsname = 'SIR'
t = np.arange(0, 100, 1)
r0 = 1
y = scipy.integrate.odeint(dY, np.array([10000-r0, r0, 0]), t)
df = pd.DataFrame(y, columns=['S', 'Positive', 'R'], index=t)
df['Negative'] = 1000#df['S']+df['R']
df['Odds'] = df['Positive']/df['Negative']
#df['Odds'] += np.random.randn(len(df))*df['Odds']*0.05
df['logodds'] =  np.log(df['Odds']) +  np.random.randn(len(df))*0.05
df['Odds'] = np.exp(df['logodds'])
df['Date'] = t
plt.scatter(df.index, df['Odds'])
plt.yscale('log')
plt.ylim([0.0005, 0.1])
plt.grid(True)
#plt.figure()
df.plot(y=['S', 'Positive', 'R'])
print((df['Odds']/(1+df['Odds'])).max())
print(1-1/1.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.052824121487497926
0.23076923076923084


In [4]:
class FirstInChunkSelector(object):
    '''Selects first element from each non zero chunk.'''

    def __init__(self, clf):
        self.clf = clf
        self.coef = None
        self.mask = None

    def select_coef(self):
        n_features = len(self.clf.coef_)
        no_zero = np.zeros(n_features+1)
        no_zero[1:] = self.clf.coef_ != 0
        self.mask = np.diff(no_zero)>0
        self.coef = self.clf.coef_[self.mask]
        return self.coef

    def transform(self, X):
        self.select_coef()
        return X[:, self.mask]

    def get_support(self):
        self.select_coef()
        return self.mask

    def get_number_of_features(self):
        self.select_coef()
        return sum(self.mask)


class LassoICSelector(object):
    """LASSO regression with FirstInChunk selector."""

    def __init__(self, X, y, criterion):
        self.lasso = linear_model.LassoLars(alpha=0, max_iter=100000)
        self.criterion = criterion
        self.selector = FirstInChunkSelector(self.lasso)
        self.OLS = sm.OLS
        self.ols = self.OLS(y, X)
        self.ols_results = None
        self.X = X
        self.y = y
        self.final_ols = False

    def transform_to_ols(self, X):
        '''Selects only the features of X are used by OLS.
        Also, adds a coloumn with ones for the intercept.
        '''

        X_new = self.selector.transform(X)
        if self.final_ols:
            X_new = X[:, self.support]
        X_new_with_cte = np.hstack([X_new, np.ones((X_new.shape[0], 1))])
        return X_new_with_cte

    def fit(self, X, y):
        '''Selects features and fits the OLS.'''

        # select features
        X_new = self.transform_to_ols(X)

        # fit ols
        self.ols = self.OLS(y, X_new)
        self.ols_results = self.ols.fit()

        # remove non signicative variables and fit again
        mask = self.ols_results.pvalues < 0.05 / len(self.ols_results.pvalues)
        Xnew = self.transform_to_ols(X)
        Xnew = Xnew[:, mask]
        self.support = self.selector.get_support()
        self.ols = self.OLS(y, Xnew)
        self.ols_results = self.ols.fit()
        while any(self.ols_results.pvalues >= 0.05 / len(self.ols_results.pvalues)):
            mask.values[mask.values] = (self.ols_results.pvalues < 0.05 / len(self.ols_results.pvalues)).values
            Xnew = self.transform_to_ols(X)
            Xnew = Xnew[:, mask]
            self.support = self.selector.get_support()
            self.ols = self.OLS(y, Xnew)
            self.ols_results = self.ols.fit()
        self.support[self.support] = mask[:-1]

    def fit_best_alpha(self, X, y):
        '''returns the model with the lowst cirterion.'''

        self.lasso.fit(X, y)
        alphas = self.lasso.alphas_
        self.criterions_ = np.zeros(len(alphas))
        self.log_liklehods = np.zeros(len(alphas))
        
        
        for i, alpha in enumerate(alphas):
            self.lasso.coef_ = self.lasso.coef_path_[:, i]
            self.fit(X, y)
            self.criterions_[i], self.log_liklehods[i] = self.get_criterion(self.ols.exog, y)
        
        # we use a list of tuples to find the minimum cirterion value.
        # If there are ties, we use the maximum alpha value.
        criterions_idx = list(zip(self.criterions_, alphas, range(len(alphas))))
        criterion, alpha, idx = min(criterions_idx, key=lambda x: (x[0], -x[1]))
        
        self.lasso.coef_ = self.lasso.coef_path_[:, idx]
        self.lasso.alpha = alpha
        self.fit(X, y)
        self.final_ols = True

    def predict(self, X):
        '''Predicts y useing the OLS fit.'''

        return self.ols.predict(self.ols_results.params, X)

    def log_liklihood(self, X, y):
        '''Computes the log liklihood assuming normally distributed errors.'''

        eps64 = np.finfo('float64').eps

        # residuals
        R = y - self.predict(X)
        sigma2 = np.var(R)

        loglike = -0.5 * len(R) * np.log(sigma2)
        loglike -= 0.5 * len(R) * np.log(2*np.pi) - 0.5*len(R) + 0.5
        return loglike

    def get_criterion(self, X, y):
        '''Computes AIC or BIC criterion.'''

        n_samples = X.shape[0]
        if self.criterion == 'aic':
            K = 2  # AIC
        elif self.criterion == 'bic':
            K = np.log(n_samples)
        else:
            raise ValueError('criterion should be either bic or aic')

        log_like = self.log_liklihood(X, y)
        df = X.shape[1]

        aic = K * df - 2*log_like
        self.criterion_ = aic

        return self.criterion_, log_like

In [5]:
# create the independent and the dependent variables
y = np.log(df['Odds'])
X = np.tri(len(y))
X = np.cumsum(X, axis=0)[:, 1:]
X = X[df.Odds.notna(), :]
y = y[df.Odds.notna()]

# create lasso instance
lics = LassoICSelector(X, y.values, 'bic')

# fit
lics.fit_best_alpha(X, y)

plt.close('all')

plt.figure()
plt.plot(lics.lasso.alphas_, lics.criterions_)
plt.scatter(lics.lasso.alphas_, lics.criterions_)
plt.vlines(lics.lasso.alpha, min(lics.criterions_), max(lics.criterions_))
plt.title('alpha={}'.format(lics.lasso.alpha))
plt.ylabel('AIC')
plt.xlabel('Alpha')
plt.xscale('log')

lics.ols_results.summary()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Odds   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.476e+04
Date:                Thu, 18 Jun 2020   Prob (F-statistic):          9.10e-132
Time:                        16:14:52   Log-Likelihood:                 160.57
No. Observations:                 100   AIC:                            -311.1
Df Residuals:                      95   BIC:                            -298.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1323      0.001     91.043      0.000       0.129       0.135
x2            -0.0637      0.002    -26.134      0.000      -0.069      -0.059
x3            -0.1061      0.002    -50.245      0.000      -0.110      -0.102
x4            -0.0436      0.001    -29.332      0.000      -0.047      -0.041
const         -6.9009      0.020   -346.724      0.000      -6.940      -6.861
==============================================================================
Omnibus:                       10.072   Durbin-Watson:                   2.456
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.317
Skew:                          -0.656   Prob(JB):                      0.00575
Kurtosis:                       3.869   Cond. No.                         305.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
data = df.copy()

#yhat = lics.ols_results.fittedvalues
y = np.log(df['Odds'])
X = np.tri(len(y))
X = np.cumsum(X, axis=0)[:, 1:]
Xols = lics.transform_to_ols(X)
yhat = lics.ols.predict(lics.ols_results.params, Xols)
# from equation 5
odds_hat = np.exp(yhat)

# the error in yhat is
#Xols = lics.transform_to_ols(X)
(yhat_std, yhat_l, yhat_u) = wls_prediction_std(lics.ols_results, Xols)

# propagation of errors
oddshat_std = odds_hat*yhat_std

data.loc[:, 'odds_hat'] = odds_hat
data.loc[:, 'oddshat_std'] = oddshat_std
data.loc[:, 'oddshat_l'] = odds_hat - 2*oddshat_std
data.loc[:, 'oddshat_u'] = odds_hat + 2*oddshat_std

# use coefficients to calculate Rt
coef = np.zeros(len(data))
coef_std = np.zeros_like(coef) * np.nan
ind = np.squeeze(np.argwhere(lics.support))

# we do not use the last coefficient since it's the intercept (=log(odds_0))
coef[ind] = lics.ols_results.params[:-1]

# using equation 2, 4 and 6
data.loc[:, 'R'] = np.cumsum(coef)/GAMMA+1

# get covarinace matrix of coefficients
cov = lics.ols_results.cov_params().values

# since the values of Rts are a sum of variables, we use the formula 
# of the sum of gaussian variables with a known covariance matrix 
stds = [np.sqrt(cov[:n, :n].sum()) for n in range(1, cov.shape[0])]

coef_std[ind] = stds

# error propagation formula
data.loc[:, 'Rstd'] = coef_std / GAMMA

data['Rstd'] = data['Rstd'].fillna(method='ffill')
data['R_l'] = data['R'] - 2*data['Rstd']
data['R_u'] = data['R'] + 2*data['Rstd']

r_index = data.R.diff() != 0
Rts = data.loc[r_index, ['R', 'R_l', 'R_u']]
display(Rts)

,R,R_l,R_u
0,1.992245,1.970448,2.014042
20,1.514720,1.495907,1.533533
39,0.719285,0.703235,0.735334
59,0.392354,0.383783,0.400925


In [7]:
plt.close('all')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[8, 4])

ax = data.plot(x='Date', y='R', legend=False, ax=axes[0])
ax.fill_between(data.index, data['R_u'], data['R_l'],
                facecolor='blue', alpha=0.2, label='95% CI')


ax.set_ylabel('Rt')
ax.grid(True)
plt.tight_layout()

#plt.savefig('../figs/{}_RtL1.jpg'.format(dsname), dpi=300)
#plt.show()

ax = sns.scatterplot(x='Date', y='Odds', data=data, label='Data', ax=axes[1])
ax = sns.lineplot(x='Date', y='odds_hat', label='Fit', ax=ax, data=data)
ax.fill_between(data.index, data['oddshat_l'],
                data['oddshat_u'],
                facecolor='blue', alpha=0.1, label='95% CI')

ax.legend()
ax.set_yscale('log')

#ax.xaxis.set_major_locator(locator)
#ax.xaxis.set_major_formatter(formatter)

plt.ylabel('Odds')
plt.grid(True)
plt.tight_layout()
ax.set_xlim(data['Date'].min(), data['Date'].max())
plt.savefig('../figs/{}_OddsL1.jpg'.format(dsname), dpi=300)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.close('all')
fig, axes = plt.subplots(ncols=2, figsize=[10, 5])
ax = data.plot(x='Date', y='R', legend=False, ax=axes[1])
axes[1].fill_between(data.index, data['R_u'], data['R_l'],
                facecolor='blue', alpha=0.2, label='95% CI')
if dsname=='New York':
    axes[1].vlines(events, 0, data.R_u.max(), linestyle='--')

axes[1].set_ylabel('$R_t$')
plt.grid(True)

ax = sns.scatterplot(x='Date', y='Odds', data=data, label='Data', ax=axes[0])
ax = sns.lineplot(x='Date', y='odds_hat', label='Fit', ax=axes[0], data=data)
ax.fill_between(data.index, data['oddshat_l'],
                data['oddshat_u'],
                facecolor='blue', alpha=0.1, label='95% CI')

ax.legend()
axes[0].set_ylabel('Odds')

axes[0].set_yscale('log')
ax.grid(True)
plt.tight_layout()
plt.savefig('../figs/{}_OddsL1.jpg'.format(dsname), dpi=300)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …